## Functions

In [ ]:
# Initial conditions
class InitialConditions_SS (UserExpression):
    def eval(self, values, x):
        values[0] = ref[0]
        values[1] = ref[1]
        values[2] = ref[2]
        values[3] = ref[3]
    def value_shape(self):
        return (4,)
class InitialConditions_SS_perturbed (UserExpression):
    def eval(self, values, x):
        amplitude1 = (eigenvector[0]).real
        amplitude2 = (eigenvector[1]).real
        amplitude3 = (eigenvector[2]).real
        amplitude4 = (eigenvector[3]).real
        values[0] = ref[0] + amplitude1*cos (-wavenumber_fourier *x[0]) 
        values[1] = ref[1] + amplitude2*cos (-wavenumber_fourier *x[0]) 
        values[2] = ref[2] + amplitude3*cos (-wavenumber_fourier *x[0]) 
        values[3] = ref[3] + amplitude4*cos (-wavenumber_fourier *x[0]) 
    def value_shape(self):
        return (4,)

# Boundary conditions
if elementspace == 4: 
    parameters ["ghost_mode"] = "shared_facet"
    # https://fenicsproject.org/docs/dolfin/2016.2.0/python/demo/documented/subdomains-poisson/python/documentation.html
    # Inflow Dirichlet boundary condition
    class Inflow (SubDomain):
        def inside (self, x, on_boundary): 
            return (x[0] < DOLFIN_EPS) and on_boundary #near
    # Ouflow Dirichlet boundary condition
    class Outflow (SubDomain):
        def inside (self, x, on_boundary): 
            return (x[0] > (L - DOLFIN_EPS)) and on_boundary #near
    # Initialize sub-domain instances
    inflow = Inflow ()
    outflow = Outflow ()
else:
    # Inflow Dirichlet boundary condition
    def Inflow (x, on_boundary): 
        return (x[0] < DOLFIN_EPS) and on_boundary
    # Ouflow Dirichlet boundary condition
    def Outflow (x, on_boundary): 
        return (x[0] > (L - DOLFIN_EPS)) and on_boundary

# Periodic boundary conditions
class PeriodicBoundary(SubDomain):
    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        return bool (x[0] < DOLFIN_EPS and x[0] > - DOLFIN_EPS and on_boundary)
        # return bool (x[0] < (l + DOLFIN_EPS) and x[0] > (l - DOLFIN_EPS) and on_boundary)
    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
          y[0] = x[0] - l
        #   y[0] = l - x[0]

# Discretization
## Spatial discretization

In [ ]:
# Define mesh
l_fenics = Constant (L)
mesh = IntervalMesh (nx, 0, l_fenics)
coordinates = mesh.coordinates ()
mesh_cells = mesh.cells () # print (mesh_cells)

# Delta s
h = CellDiameter (mesh) # Cell minimum size
# deltax = L/nx
deltax = h

if elementspace == 4:  
    # Mesh-related functions
    n = FacetNormal (mesh)
    # h = CellDiameter (mesh)
    h_avg = (h ("+") + h ("-"))/2

    # Initialize mesh function for boundary domains
    boundaries = MeshFunction ("size_t", mesh, mesh.topology ().dim () - 1, 0)

    # boundaries = MeshFunction ("size_t", mesh)
    boundaries.set_all (0)
    inflow.mark (boundaries, 1)
    outflow.mark (boundaries, 2)

    # Save sub domains to file
    file_boundaries << boundaries

## Time discretization

In [ ]:
if any ([discretization == 2]):
    # Define time discretization properties
    if CFL_velocity == "liquid":
        u_CFL = ref[1]
    elif CFL_velocity == "gas":
        u_CFL = ref[2]
    elif CFL_velocity == "max_velocity":
        u_CFL = max (ref[1], ref[2])
    elif CFL_velocity == "max_characteristic":
        u_CFL = maxchar

    # Time step input
    if timestep == "timestep_input":
        dt = timestep_in
    elif timestep == "cfl_based":
        dt = CFL*deltax/(u_CFL)  # Van Zwieten & Smith /lambda_max Holmas (maximum modulus of the eigenvalues of the Jacobian of (4.1) at time 

    # Final time input
    if T_simulation == "T_input":     # OPTIONS: T_cfl_based, T_input
        T = T_in
        num_steps = round (T/dt)
    elif T_simulation == "T_steps_based":
        T = num_steps*dt

    # Print info
    if show_data == 1:
        print ("INFO: T = ", T)
        print ("INFO: dt = ", dt)
        print ("INFO: steps = ", num_steps)

## Element spaces

In [ ]:
# Define element spaces 
# https://fenicsproject.org/docs/dolfin/1.5.0/python/programmers-reference/functions/functionspace/FunctionSpace.html#dolfin.functions.functionspace.FunctionSpace
if elementspace == 1:
    # Element spaces
    e1 = "Lagrange" 
    e2 = "Lagrange"
    e3 = "Lagrange"
    e4 = "Lagrange"
    # Degree of basis functions
    deg1 = p
    deg2 = p
    deg3 = p
    deg4 = p

elif elementspace == 2:
    # Element spaces
    e1 = "Lagrange" 
    e2 = "Lagrange"
    e3 = "Lagrange"
    e4 = "Lagrange"
    # Degree of basis functions
    deg1 = p - 1
    deg2 = p
    deg3 = p
    deg4 = p - 1

elif elementspace == 3:
    # Element spaces
    e1 = "Lagrange" 
    e2 = "Lagrange"
    e3 = "Lagrange"
    e4 = "Lagrange"
    e2_b = "Bubble"
    e3_b = "Bubble"
    # Degree of basis functions
    deg1 = 1 
    deg2 = 1
    deg3 = 1
    deg4 = 1
    deg2_b = mesh.topology ().dim () + 1
    deg3_b = mesh.topology ().dim () + 1

elif elementspace == 4:
    # Element spaces
    e1 = "DG"
    e2 = "DG"
    e3 = "DG"
    e4 = "DG"
    # Degree of basis functions
    deg1 = p
    deg2 = p
    deg3 = p
    deg4 = p
    # Penalty terms
    alpha1 = Constant (4.0)
    alpha2 = Constant (4.0)
    alpha3 = Constant (4.0)
    alpha4 = Constant (4.0)

    gamma1 = Constant (8.0)
    gamma2 = Constant (8.0)
    gamma3 = Constant (8.0)
    gamma4 = Constant (8.0)

# Define elements and mixed elements
if any ([elementspace == 1, elementspace == 2, elementspace == 4]):
    # Define function space
    element1 = FiniteElement (e1, mesh.ufl_cell (), degree = deg1)
    element2 = FiniteElement (e2, mesh.ufl_cell (), degree = deg2)
    element3 = FiniteElement (e3, mesh.ufl_cell (), degree = deg3)
    element4 = FiniteElement (e4, mesh.ufl_cell (), degree = deg4)
    # Mixed element
    element = MixedElement ([element1, element2, element3, element4])
elif elementspace == 3:
    # Define function space
    # Continuous elements
    element1 = FiniteElement (e1, mesh.ufl_cell (), degree = deg1)
    element2 = FiniteElement (e2, mesh.ufl_cell (), degree = deg2)
    element3 = FiniteElement (e3, mesh.ufl_cell (), degree = deg3)
    element4 = FiniteElement (e4, mesh.ufl_cell (), degree = deg4)
    # Bubble elements
    element2_b = FiniteElement (e2_b, mesh.ufl_cell (), degree = deg2_b)
    element3_b = FiniteElement (e3_b, mesh.ufl_cell (), degree = deg3_b)
    # Mixed element
    element = MixedElement ([element1, NodalEnrichedElement (element2, element2_b), NodalEnrichedElement (element3, element3_b), element4])

# Print degree of basis functions
if show_data == 1:
    print ("INFO: elementspace = ", elementspace)
    print ("INFO: deg1 = ", deg1)
    print ("INFO: deg2 = ", deg2)
    print ("INFO: deg3 = ", deg3)
    print ("INFO: deg4 = ", deg4)

## Functions

In [ ]:
# Define function space
if dirichlet_type == 2:
    V = FunctionSpace (mesh, element)
elif dirichlet_type == 1:
    pbc = PeriodicBoundary ()
    V = FunctionSpace (mesh, element, constrained_domain = pbc)    
    #boundaries
    # https://fenicsproject.discourse.group/t/dg-for-advection-equation-with-periodic-boundary-condition/272

# Define variational problem
dvariable = TrialFunction (V)
v1, v2, v3, v4 = TestFunctions (V)

# Define functions
variable = Function (V) # solution from (n+1) current step
variable_n = Function (V) # solution from (n) previous converged step
variable_n2 = Function (V) # solution from (n) previous converged step
variable_past = Function (V) # solution from (n-1) previous converged step
variable_ic = Function (V) # reference solution from initial conditions

# Split mixed functions
dvariable1, dvariable2, dvariable3, dvariable4 = split (dvariable)
variable1, variable2, variable3, variable4 = split (variable)
variable1_n, variable2_n, variable3_n, variable4_n = split (variable_n)
variable1_n2, variable2_n2, variable3_n2, variable4_n2 = split (variable_n2)
variable1_past, variable2_past, variable3_past, variable4_past = split (variable_past)
variable1_ic, variable2_ic, variable3_ic, variable4_ic = split (variable_ic)